## Neural Network

In [3]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [113]:
path = r"C:\Users\18110\OneDrive\Desktop\Train\Train"

data = []
Y = []

classes = ["Jade" , "James" , "Jane" , "Joel" , "Jovi"]
label_map = {name: i for i, name in enumerate(classes)}

for sub_file_name in classes:
    sub_file_path = os.path.join(path , sub_file_name) #getting path subfiles

    for img_name in os.listdir(sub_file_path):
        img_path = os.path.join(sub_file_path , img_name) #getting path of images in subfiles
        
        img = cv2.imread(img_path , cv2.IMREAD_UNCHANGED) #getting the image
        img = cv2.resize(img , (28,28)) #resizing
        img = img/255.0 #normalizing pixel values to 0 and 1 

        data.append(img)
        Y.append(label_map[sub_file_name])
data = np.array(data) #shape is (4000 ,28 ,28)
data = data.reshape(data.shape[0] , -1)

pixel_columns = [] 
for i in range(data.shape[1]):
    pixel_columns.append(f"pixel_{i}")

df = pd.DataFrame(data , columns = pixel_columns)
df["label"] = Y
df.head()
data

array([[0.        , 0.        , 0.        , ..., 0.00784314, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00784314, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00784314, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00784314, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00784314, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00784314, 0.        ,
        0.        ]], shape=(4000, 784))

In [116]:
m,n = df.shape # m = 4000 , n = 784
df = np.array(df)
np.random.shuffle(df)
df_train = df.T
X_train = df_train[0:n-1]
Y_train = np.array(Y)
Y_train = Y_train.T
X_train.shape

(784, 4000)

In [133]:
def initial_params():
    W1 = np.random.randn(5 , 784) # for first hidden layer
    b1 = np.random.randn(5 , 1)
    W2 = np.random.randn(5 , 5) #for second hidden layer
    b2 = np.random.randn(5 , 1) 
    return  W1 , b1 , W2 , b2

def ReLU(Z):
    return np.maximum(0 , Z)
    
def ReLU_der(Z):
    return Z > 0

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))
    
def forward_prop(W1 , b1 , W2 , b2 , X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1 , A1 , Z2 , A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size , Y.max() + 1))
    one_hot_Y[np.arange(Y.size) , Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = (1/m) * dZ2.dot(A1.T)
    db2 = (1/m) * np.sum(dZ2 , 1)
    dZ1 = W2.T.dot(dZ2) * ReLU_der(dZ2)
    dW1 = (1/m) * dZ1.dot(X.T)
    db1 = (1/m) * np.sum(dZ1 , 1)
    return dW1 , db1 , dW2 , db2
    
def update_params(W1 , b1 , W2 , b2 , dW1 , db1 , dW2 , db2 , alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * np.reshape(db1 , (5,1))
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * np.reshape(db2 , (5,1))
    return W1 , b1 , W2 , b2

def get_predictions(A2):
    return np.argmax(A2 , 0)

def get_accuracy(predictions , Y):
    Y_labels = np.argmax(Y , axis = 0)
    print(predictions , Y)
    return np.sum(predictions == Y) / Y.size

def gradient_des(X ,Y , iterations , alpha):
    W1 , b1 , W2 , b2 = initial_params()
    for i in range(iterations):
        Z1 , A1 , Z2 , A2 = forward_prop(W1 , b1 , W2 , b2 , X)
        dW1 , db1 , dW2 , db2 = backward_prop(Z1 , A1 , Z2 , A2 , W2, X, Y)
        W1 , b1 , W2 , b2 = update_params(W1 , b1 , W2 , b2 , dW1 , db1 , dW2 , db2 , alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
        return W1 , b1 , W2 , b2
        

In [134]:
W1 , b1 , W2 , b2 = gradient_des(X_train , Y_train , 500 , 0.001)

Iteration:  0
[2 2 2 ... 2 2 2] [0 0 0 ... 4 4 4]
Accuracy:  0.203
